In [140]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))


In [1]:
import xgboost as xgb

In [4]:
# filepath = 'C:/dev/csb-net/extracted/reCurse/2843387/frames_reCurse_2843387.csv'
filepath = 'C:/dev/csb-net/parsed/aelyanne/2697232/frames_aelyanne_2697232.csv'

import math
import numpy as np
import pandas as pd

df = pd.read_csv(filepath, index_col=False)
print('Columns:', df.columns)
#df.describe()
# df.head()

def augment(df, orig_data, dx, dy):
    x_cols = ['p1x', 'p2x', 'p3x', 'p4x', 'p1wx', 'p2wx', 'p3wx', 'p4wx', 'p1tx', 'p2tx']
    y_cols = ['p1y', 'p2y', 'p3y', 'p4y', 'p1wy', 'p2wy', 'p3wy', 'p4wy', 'p1ty', 'p2ty']
    translated = df2.copy()
#     print('translated\n', translated.head())
    translated.loc[:, x_cols] += dx
    translated.loc[:, y_cols] += dy
    print(dx, dy, df.shape)
#     print('translated\n', translated.head())
    return pd.concat([df, translated])

# print(df.shape)
# print(df.tail())

orig_data = df.copy()

# The walrus operator is new in python 3.8 
# [df := augment(df, orig_data, dx, dy) for dx in range(-150, 150, 100) for dy in range(-150, 150, 100)] 

print(df.shape)
print(df.tail())

y = pd.concat([df.pop(x) for x in ['p1tx', 'p1ty', 'p1t', 'p2tx', 'p2ty', 'p2t']], 1)
X = df
print(y.head())

from sklearn import preprocessing

y[['p1tx', 'p1ty']] = preprocessing.normalize(y[['p1tx', 'p1ty']], norm='l2')
y[['p2tx', 'p2ty']] = preprocessing.normalize(y[['p2tx', 'p2ty']], norm='l2')

print(y.head())
# print(X.head())


Columns: Index(['p1x', 'p1y', 'p1sx', 'p1sy', 'p1a', 'p1wx', 'p1wy', 'tf1', 'pf1',
       'p2x', 'p2y', 'p2sx', 'p2sy', 'p2a', 'p2wx', 'p2wy', 'tf2', 'pf2',
       'p3x', 'p3y', 'p3sx', 'p3sy', 'p3a', 'p3wx', 'p3wy', 'tf3', 'pf3',
       'p4x', 'p4y', 'p4sx', 'p4sy', 'p4a', 'p4wx', 'p4wy', 'tf4', 'pf4',
       'p1tx', 'p1ty', 'p1t', 'p2tx', 'p2ty', 'p2t'],
      dtype='object')
(20687, 42)
        p1x   p1y  p1sx  p1sy       p1a   p1wx  p1wy  tf1  pf1   p2x  ...  \
20682  8090  1724   306   134  0.881515  10310  3336    1    1  7828  ...   
20683  8463  1932   317   176  0.833732  10310  3336    1    1  7655  ...   
20684  8850  2179   329   209  0.790177  10310  3336    1    1  7499  ...   
20685  9251  2458   340   237  0.773387  10310  3336    1    1  7365  ...   
20686  9662  2765   349   260  0.775194  10310  3336    1    1  7258  ...   

        p4wx  p4wy  tf4  pf4   p1tx  p1ty  p1t  p2tx  p2ty  p2t  
20682  11176  5442    0    0   6378  4008  100  7153  6313  100  
20683   7261

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

print(f'Train data contains {X_train.shape[0]} sample')
print(f'Test  data contains {X_test.shape[0]} sample')

Train data contains 16549 sample
Test  data contains 4138 sample


In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

sc_inputs = StandardScaler()
X_train_std = sc_inputs.fit_transform(X_train)
X_test_std = sc_inputs.transform(X_test)

# sc_outputs = StandardScaler()
sc_outputs = MinMaxScaler(feature_range = (-1, 1))
y_train_std = sc_outputs.fit_transform(y_train)
y_test_std = sc_outputs.transform(y_test)

print(y_train)
print(y_train_std)

X_train_std, X_val_std, y_train_std, y_val_std = train_test_split(X_train_std, y_train_std, test_size=0.1, random_state=1)

           p1tx      p1ty  p1t      p2tx      p2ty  p2t
3303   0.948578  0.316544  100 -0.734000 -0.679150  100
2233   0.337634  0.941278  100  0.379908  0.925024  100
6845   0.965248  0.261336  100 -0.085759  0.996316  100
14282  0.952684  0.303963  100  0.544592  0.838701  100
17030  0.978194  0.207693  100  0.899103  0.437738  100
...         ...       ...  ...       ...       ...  ...
1099  -0.702701 -0.711486  100 -0.893773 -0.448520  100
18898 -0.999977  0.006836  100  0.980575 -0.196145  100
11798  0.997020  0.077146  100  0.994428  0.105421  100
6637   0.555769  0.831337  100 -0.911374  0.411579  100
2575   0.991163  0.132646  100 -0.750063 -0.661366  100

[16549 rows x 6 columns]
[[ 0.94857788  0.31654403  1.         -0.73399982 -0.67914966  1.        ]
 [ 0.33763358  0.94127763  1.          0.37990829  0.9250242   1.        ]
 [ 0.96524794  0.26133608  1.         -0.08575884  0.99631597  1.        ]
 ...
 [ 0.99701987  0.0771462   1.          0.99442764  0.10542146  1.       

In [29]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV

multioutputregressor = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror')).fit(X_train_std, y_train_std)

parameters = {'estimator__objective':['reg:squarederror'],
              'estimator__learning_rate': [.03, 0.05, .07], #so called `eta` value
              'estimator__max_depth': [5, 6, 7, 8],
              'estimator__min_child_weight': [4],
#               'silent': [1],
              'estimator__subsample': [0.7],
              'estimator__colsample_bytree': [0.7],
              'estimator__n_estimators': [500]}

xgb_grid = GridSearchCV(estimator=multioutputregressor,
                        param_grid=parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train_std, y_train_std)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)


c:\users\francois\appdata\local\programs\python\python38\lib\site-packages\xgboost\data.py:94: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  24 out of  24 | elapsed: 12.0min finished
c:\users\francois\appdata\local\programs\python\python38\lib\site-packages\xgboost\data.py:94: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


-0.16363729433104396
{'estimator__colsample_bytree': 0.7, 'estimator__learning_rate': 0.03, 'estimator__max_depth': 8, 'estimator__min_child_weight': 4, 'estimator__n_estimators': 500, 'estimator__objective': 'reg:squarederror', 'estimator__subsample': 0.7}


In [30]:
from numpy import array, sort

model = multioutputregressor
model = xgb_grid.best_estimator_

y_pred_std = model.predict(np.array(X_test_std))

y_pred = sc_outputs.inverse_transform(y_pred_std)

# print(y_pred)
# print(y_test)
print('Delta\n', y_pred - y_test)



Delta
            p1tx      p1ty       p1t      p2tx      p2ty       p2t
10563 -0.014489  0.121990 -0.000626  1.133398  0.538253  0.004784
2968   0.013732 -0.061867 -0.000771  0.772507 -0.364679  0.006172
4985   0.659604 -0.223951 -0.010315 -0.055596 -0.206733  0.009010
5694  -0.014210  0.058282  0.000175 -0.074640 -0.071851 -0.001511
2394   0.598214 -0.230155 -0.002647  0.156848 -0.129258  0.151466
...         ...       ...       ...       ...       ...       ...
1657   0.196765 -0.259271  0.000191 -0.557925  0.173931  0.045326
5377   0.673907  0.190470 -0.000130  0.680911  0.386546 -1.719650
6156  -0.073640 -0.010187  0.000473 -0.186037 -0.018173  0.010490
3328  -0.368400 -0.202561 -0.000557  0.230840 -0.211330  0.016258
2303  -0.371838  0.062362  0.000015 -0.096596  0.053029  0.025726

[4138 rows x 6 columns]


In [31]:
R = model.score(X_train_std, y_train_std)
print('R2 train', R)

R = model.score(X_test_std, y_test_std)
print('R2 test', R)


R2 train 0.9351467878480033
R2 test 0.7127078014898913


In [21]:
y_test_pred_std = model.predict(X_test_std)

y_test_pred = sc_outputs.inverse_transform(y_test_pred_std)

delta = y_test - y_test_pred
print(y_test)
print(pd.DataFrame(y_test_pred))
print(np.square(delta))

           p1tx      p1ty  p1t      p2tx      p2ty  p2t
10563  0.870610  0.491975  100 -0.856305 -0.516471  100
2968  -0.546403  0.837522  100 -0.435981  0.899956  100
4985  -0.611282  0.791413  100 -0.456292  0.889830  100
5694   0.992731  0.120353  100  0.986380  0.164482  100
2394  -0.660889  0.750484  100 -0.402040  0.915622  100
...         ...       ...  ...       ...       ...  ...
1657   0.695537  0.718490  100  0.732549  0.680714  100
5377  -0.977610  0.210427  100 -0.987613 -0.156908  100
6156   0.886283  0.463144  100  0.891593 -0.452838  100
3328   0.664333  0.747437  100  0.680320  0.732915  100
2303   0.970990  0.239122  100  0.969172  0.246387  100

[4138 rows x 6 columns]
             0         1      2         3         4          5
0     0.756376  0.657234  100.0 -0.264266 -0.169329  99.999893
1    -0.491448  0.687859  100.0 -0.027087  0.524149  99.999893
2    -0.397790  0.518149  100.0 -0.527344  0.739630  99.999893
3     0.998678  0.089372  100.0  0.957910  0.052055

In [78]:
print(y_test_pred[0])
print(y_test.iloc[0,:])

[-59890.98     66033.42       142.58134 -49680.047   -63080.
    115.31177]
p1tx   -49844
p1ty    84861
p1t       200
p2tx   -68361
p2ty   -64256
p2t         0
Name: 12970, dtype: int64


In [62]:
i = 1

printmd('**input**')

p = frames[i].pods[0].position
print('pos:', int(p.x * 16000), int(p.y * 9000))

s = frames[i].pods[0].speed
print('speed:', int(s.x * (650 + 200)), int(s.y * (650 + 200)))

a = frames[i].pods[0].angle
print('angle:', int(a * 180))

w = frames[i].pods[0].waypoint
print('waypoint:', int(w.x * 16000), int(w.y * 9000))

printmd('**output**')

t = frames[i].actions[0].aim
print('aim:', int(t.x * 1000), int(t.y * 1000))
#print('aim abs:', int(p.x * 16000) + int(t.x * 1000), int(p.y * 9000) + int(t.y * 1000))
pw = frames[i].actions[0].power
print(int(pw * 200))

t = frames[i].actions[1].aim
print('aim:', int(t.x * 1000), int(t.y * 1000))
#print('aim abs:', int(p.x * 16000) + int(t.x * 1000), int(p.y * 9000) + int(t.y * 1000))
pw = frames[i].actions[1].power
print(int(pw * 200))

printmd('**prediction**')

p = model.predict(np.array(inputs)[i:i + 1])
bot = p[0]
print(bot)
print(int(bot[0] * 1000))
print(int(bot[1] * 1000))
print(int(bot[2] * 200))
print(int(bot[3] * 1000))
print(int(bot[4] * 1000))
print(int(bot[5] * 200))

**input**

NameError: name 'frames' is not defined

In [48]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 30)                750       
_________________________________________________________________
dense_7 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_8 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_9 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 44        
Total params: 2,044
Trainable params: 2,044
Non-trainable params: 0
_________________________________________________________________


In [27]:
import pickle

a = [x.get_weights() for x in model.layers]

#a = model.layers[0].get_weights()
serialized = pickle.dumps(a, protocol=0) # protocol 0 is printable ASCII

print(serialized)


#for layer in model.layers:
#    print(layer.get_weights())


b'(lp0\n(lp1\ncnumpy.core.multiarray\n_reconstruct\np2\n(cnumpy\nndarray\np3\n(I0\ntp4\nc_codecs\nencode\np5\n(Vb\np6\nVlatin1\np7\ntp8\nRp9\ntp10\nRp11\n(I1\n(I28\nI30\ntp12\ncnumpy\ndtype\np13\n(Vf4\np14\nI0\nI1\ntp15\nRp16\n(I3\nV<\np17\nNNNI-1\nI-1\nI0\ntp18\nbI00\ng5\n(V\\u000a"@>\x18&\xd2=M\\u0000\xb7=M\x1b=\xbd\x94\x92 \xbe\xf0\x91\xe3\xbd#\x87\'>\x8b7\x91>\xfc\x0fP\xbcWX\x98=Z\x0b6\xbf\xb0P\x89<\x11B\xa4;K\xb1#=#B\x1c\xbe\x9e$\x19\xbf\x10\xf7\xa5=\xf2\x12\x1d<q"i=\xdf"\x84>\x8e4?\xbd3\xea\xb8<E\xdfh\xbb\x1d\x83\xf7=\x8d\x1dH\xbe\xa4\x04l>\xe3\x99><\xa9F\t;Dgq\xbe\xb28\\u001a\xbe\x91I\x8f\xbdS\xc1\x80>+9\x9b=\xa4>X\xbe\\u000a\xeec\xbc\x95\xbf\xb2\xbd\xca\xfb6>\xdd|\x11\xbe\x89h\xf0\xbd\xbe\x14\xb0\xbd\x8f\xd6\x81\xbejh;\xbc\xa9N\x04\xbe\x8c>/\xbe\xb0\xfdb\xbeva\xd0\xbe$\x80\xfe\xbdi\xac\x83\xbcd[\x07=n\x05\x03=\x97\x89\x1d=\xf0+\xc8\xbd\x9d\x84:>\x03(Y>\xcf\xdd\x97\xbe\xdb\x1f\x81>\x84\x9a\x0b>J\x18\x04\xbdx\x1e\'>u\xc1x\xbd\xf3\x9f\xf3=-[\x86=\xecG\xe1=\x95\x0e\x06>\xa5\xb1\x96

In [31]:
data = b'(lp0\n(lp1\ncnumpy.core.multiarray\n_reconstruct\np2\n(cnumpy\nndarray\np3\n(I0\ntp4\nc_codecs\nencode\np5\n(Vb\np6\nVlatin1\np7\ntp8\nRp9\ntp10\nRp11\n(I1\n(I28\nI20\ntp12\ncnumpy\ndtype\np13\n(Vf4\np14\nI0\nI1\ntp15\nRp16\n(I3\nV<\np17\nNNNI-1\nI-1\nI0\ntp18\nbI00\ng5\n(V\xf1\xe80\xbf\x0e\xb2\x9a?\xfd\xea\xa8=\x9a\xec\x89?\xc1\xd6\x06\xbe},>@\xa2p,\xbd<\xc3\x80\xbd\xb2\x11\\u0000\xbf\xe2\x8fI\xbeq\xc4\x98\xbe\xc8gK>]9\t?\xeaS\xe2>\xf9\xdb\x81\xbe\xd1(3<\x80\xe2a\xbc4\xf14\xbe\xde;\x9b\xbd\xd3$\x16\xbfQy\xdf>}D\xe2>?M\xb9>\xf3_\x88<W\xd9\\u0000\xbd\xe1U\xd9=\xd3\x02\x9c>\xee\x1c\xd4<D\xe5/\xbf\x08\x8c\xb7\xbd\xbb4\x18\xbd\xcb\xa7Z\xbe\'i\xb4\xbe\xe8^\x8f>0\x86_\xbe\xb1n\x18>\xe7\xc2\x9d\xbd\x81M.\xbe\xaa\xf0d\xbc\x84\\u001a,\xbfp\x15\xde\xbdV\xff\xee>"\xb8\\u000a?\x80\xea\x9d\xbfs_\xdf\xbdP\xd4F\xc0\xf7v4\xbe\xff\xc4\x9a>\x91\x1f[>\x08\xd8 =\x87\xd2\x10>\tt\x91<\xd0\x15\xa3>\x97$Q\xbf\xd0\xc6)\xbf\x90\xb2k\xbdh1b\xbe\x05\\u005cl>\xcd\x04\x88\xbeX\xd9\xde\xbc\x7f\xa3\x02\xbfT\xccw\xbd1\x88\x8f>\x95\xc5\xca>)\x86\x82=Q\\u005c\x03\xbfu\x14\x8b=rJ*>\xb9\xad\x86\xbe_+\xd4<\xcf8\x8d=\x1b"\x14>\xf9\xb8+\xbfG\xe9\xa4=\xce\xb9e\xbe~\xa3,\xbe\x93\xc6\x0f?&\xcb\x04?\xd5\x0e\xce>\x8e\x88T\xbe\x02\xe0\xb0?_=0?\xca\x07\xa2\xbfh\xcfa>$\xc3\x13\xbe\\u001a\xb2\x92\xben\xe2d\xbb\xd3\xb4\x80?p\xe5\xfa\xbd\xf1\x9eh\xbd\xd5\xe0\xa6>\xdeT\xbe\xbdk:B\xc0D8\xe4>\xcfoI\xbfp(W=Y&\xa4\xc0\xa9M\xa5\xc0o\xab\xb5\xbf\xfdN\x08\xc0oI9?\x0eT\x14>V\xd9\xbb>\x16\x9d\xc0\xbeM\xba\x8e\xbe\xb5\xc5\x9e\xbf\x9e\xf1 \xbc\xe4\x8eG\xbfi2\xc6>\x17\xeaB\xbe\xfaF\x96>q\x81\x83\xbe\x13\xddL\xbfJyU\xbeyW\x18>\xed\xfa\x86==\xe8\x11>f\x9fC>X1_\xbe\x18\\u0000\xbc>T\x9b>>\xb3~[>EkG\xbe\x9b \xc2>\x98\xa5\x93\xbeCp\x9d=\x87d\xb1\xbd\xbd\xef\x8d=i\xa3\xc8>\xbd\xde\xe8\xbd\xc8J \xbehE\x83\xbe?\xae\x1b==\xf0+\xbf\x89\x06\\u001a>\xceS\xf9>X|\x91=\x7f\x9d\x98<\x8a\x182>fz\x1d?\x92\xf8\xbe=/&#?\xcd$K\xbd\xd0\x89\xcd\xbe\x8f\xcb\x7f\xbd\x07\x8c\x03?\\u000d\xf9\x06?cj\x9e\xbeN\xea4\xbf\xe0\xd6\xa4\xbey\x9c\xbf\xbd\xd1\xce\xc3=\\u000az\x11\xbe\x0cy\\u000a\xbf\xd6\xb8\x0b>\xb5\xaaD\xbe\xf0\xd7L>\x1e\x96w=\xccR&?p"\x8a\xbe\xf8\x15W?\\u005c\xcfI\xbeT\x1eH>~\x10:?\xf8\x95z=\xcam\x04\xbd\x1e\x11!>h\x8b\xbb=1+*>k\xb2Z\xbd\xb4\x80M=,\xc4\xcc\xbe\xf6Xc\xbc\xf8"\x12\xbcn\xa6c>\xaeP\xbd=\xea4Q\xbd\xdeL\x93\xbd\xc2\x06P>{T\xb3\xbbT/\xd2>\x0c\xcaQ>\xdd\x10n>\xd2\xc4\xa3>UU#?\xde\x0e\x0e>h\xac\xfd>\xb5,r\xbe\xcd\xb2\xd3>\xe7\xd3\xc3>9UV\xbe\xde\xb7\x02>3S>\xbevX\xec\xbc\x9c\\u000d\x86=X<O\xbfiN\xa6=\xde\x04\xa7=uL\xda\xbc\xb4H\xa7\xbd\xe3\x8d\x83=\x9e\x87\xfd\xbd\xc4O\x08\xbdA\xe2\xd6\xbe\xaa\xa2\x87\xbe\xafe\x10?7\xcc\xc4\xbc\x11m\xb8=o t\xbf\xbdC\xf6\xbe\x15/\xa8\xbe\xb0v\x96>$\x88\x9b>b\xc8\x85\xbd\xce\x14\x01\xbe\x07\x0f{\xbe!X\xd9<\x17\xf2\x01<X}\xff=3\x9d1>,\xa3\xaf>\xf0\xdd$>\x18\x87&?\xe4\xe5\x1d\xbcs\x18\xae\xc0\xb4\xb0\\u0000?\xbb\xb5\xc3@1\x93P?8\x86\xa0@\x11]\xb2@nL\xf3\xbf\xa7N_@\x90\xa7B=^\xf0\x89\xbf\x1f\x98T?\x03\x1f\xe6@\xc7\xad\xd4\xbd\x89\x98\'\xbdeD\x81\xbeWw@\xbd..\xd6>\x82Y\x96=\x8fC\xa6>\x9a\xa5y>],w=\\u000d\x12\x1e>\xc7\xe9\xbe\xbda\xcb:>:\xea8>g\x05U>\xceG\x11>u\xcfJ\xbe\xec\x88\xf9\xbd\xd2O\xe8=\xb2\xf4\xb9<\x06\xb09?\xd8\xaa\xdb\xbd\xca%\xc6\xbd=\xdb\x9f\xbd\x98\xef\xf0=%\x1fo=\x18+\xb7>\\u000a\xce\xa7\xbd13\xc4>\x15{3\xbd7\xd0\xba>\xccQ\x82\xbd$\x1e\x14>z\xfc\xd2>[E\xc1=\x125@>7 T\xbe\x92D\xab\xbe"\x07E\xbe\xb7#\xc6>S\xf6\xb8>\xb5\xff\x1b\xbd\x06\xeb\xd8\xbc\xfe\x07\xa0\xbc\xfd\xd5\x0f\xbd\xfe\xf9\x19;\x94\xf3\xe2<(\xb5\x90\xbe:\x86t>\x94\xcbH\xbe\x99\x18 ?\x05\xbdE=y\xb7+<\xc1?\x9e\xbe:F\x1c\xbd\x94\x8d\x85>\xc3H\x8b=\\u0000A{\xbe\xe9\x916>\xd4\xeb\xc4=o\x9e\xd2<-\xeb\xa4\xbd\xeb\xee\x88\xbc*\xce\xd8\xbd\xf48\xa9>\xd2u\xb6\xbe\x99\xdd ?\xdb\xeb\x83\xbc\x95 \x96\xbe||\'\xbe\xab\x03(>\xa9\x0f\x81\xbe\xc2\t\xa6\xbd\x7f\x10\x82\xbd\xa8\xf2\xa5\xbdd\x1eY\xbe\xcb\xefs>o\xd5\x87\xbe;\xb7n\xbe\xc3\xd2\xa2\xbe\xa5L\xb0<\xf7(\x04>a\xf4\x87=A\xe4\x86>7\xcd\x10\xbdq%\xe2\xbd7Mu\xbd\x8b\x8dn=\xbe\xc4\x86\xbe\xbf\xf7\x08\xbe(=\xf4\xbe\xf2n\x18\xbe\xf6\xc6t=\x14\xf8F\xbe\x9d\x12\xeb\xbd\x13\x93\xce\xbe\xdc\xa3\x05\xbe\x17hj\xbd\xdeC\xe0\xbd\xfa\x08\x9f\xbe/\x9aS>y\xbc|=\x15\xb6V\xbb\xf3E\\u000d\xbd\xef\xad\x8e>g\x85\x12\xbf\x18<v?\x96(\x94>\xd7p\'\xbf\x908\\u001a\xbe\x10{\x1d\xbfB\x1e\x9f\xbd\x96\xa6\x12\xbe\xaf\xf6,?\x84i\\u001a\xbeYi\\u005c\xbf\xe8\xcb\xf6\xbc\xd8\x83\xb5\xbd\x8f{\xee\xbe.@\xdc\xbeSv\x7f\xbe\xafN?\xbd\xa5\xe8\x13\xbd\x93\xeb\x16\xbf\xe7u:>\xbc \xca\xbd\xd3{\xab\xbbq\xe6\x97\xbdsm\xb9\xbdZ\\u005c\x93<%\x0b\x8f>\xf3\x8a\t\xbd3:\xbc<\xbc\x82c\xbdI0m<w\x8ab>\x8f\xc3\xdd=o\xc2\x9c\xbc\xdbX(=\xf9I\xae=\x8f\\u005cB<\xd0\x8b\x94\xbb\xca\x8d\x88<@9\'>\xa1?%\xbe\x7f\xb5\xc5\xbd\xc57\xf0>|J\xb8<\xa1%{?h\xc0\xa4\xbe\x13?\x9e\xbc\xab\x18\xc3\xbe\x8e\xff/\xbe\xa5\x82v?\xa5]\'\xbe 3\xdb\xbe\xfe\xa5\\u0000\xbe.\xad\xcb\xbeU\x90\x01\xbfo1\x9a\xbe=p\xa2=\xb2Y\x11>\x98\xc4\x07<k\xae\x1b\xbe\x8e\xd3\x9b>\xf8\x8ar\xbdKR\\u000a>\x158#\xbe\xa6\xd8\xf2>\xaf\x8a\xca\xbe\x10\xd5\xd4>-\x11>\xbes\x8c\xd2\xbec\\u000a3\xbe8\\u000aJ\xbel\xe0\xc1>$7\xe9=\x02\x05K\xbe\x96\x88i\xbe\x90?\x8e<\x0fF\x8a>\xba\x03\xe9=|N\xf0<D\x86->\xf3\xd7+<\xe8N(>7\xe6H\xbeG\xbet\xbe\xb7\xfb<>Z"\xc0\xbdo\xafO\xbe=V\x81\xbd\x88C\x86\xbdC\xfc2\xbd\xc4XF\xbd\xfb]o\xbe`\xe8\x9c\xbd\xd5\xdfn\xbdQ\x96\xd1\xbdk\\u001a\x80=\xa3\\u0000\x15>\x93(c\xbc\x1b?\xb0<\x14\xf6\xa1\xbe\xf5;\xb8>\\u000dk\xf2\xbd\xa6\xad\xb4>\xa9\xef\xb6\xbe6G\xe7>K\x97<\xbe\xc8\x9bD>\x0el\x01\xbe\xe7q\x1b\xbd3\xf6\x01?U\x97\xf6\xbd\xf0\x98\xa4\xbd\xbf\xd8D>\x08\x7f\xa5\xbc\'\xea\x16\xbf\x95\x93#\xbd#\xd3T>9\xa5\x0f>\x1e\xe4\x9d<\xce\xcc\xe0\xbc\xb3\x92\xcb>h\xd14\xbe"\xe1G\xbe\xd5\x0c\xa4>\x1d\xf3\xea\xbe~8E\xbd\xcd\xc1\x83\xbfv\x11K=\xb0\xa5E>|\x9aG\xbeJ\xa2\xd4;)Ca?S\x04w>\xc4J\x9a=\x11\xbc\xaf=\xae\xa4@\xbe\x98\x8d\xf1\xbc\xfb\xd2\xa5\xbcv\xb4Y\xbbc\xc3_\xbe\xf2N\xd2\xbd\xcf\x05R\xbe!\xafT>\xdcb\xbe>i*\x97=\xfe\x02\xd6=\xde\x8f]>\x11|I;\xf8Y\xf5=\x94\xe0\x84\xbexA\x1d>`\\u000a\xe1>6\x90k\xbd\x11?\x06<q\xf2\x9a>C\xd2\x02\xbeSo\x16>\x91\x03\x11<{"|\xbdD\\u001a\xcf\xbe\xdac|\xbd\xddD\x9e\xbd>\xfc9\xbe\xbf8g\xbe\x03j\x83>\xf2m\x93\xbd\xb4\xbb\xf6\xbd\xbe{\x0b\xbdY\xec\x8d=U1\xc0>\xab\xa2\xda\xbb\x8f\x91\'\xbf\x14\xbew\xbd/\x07\x8f\xbd\x94nu\xbe\x05L\xfb\xbd\xca[\x8a\xbd\xb5\xd1\x9f\xbc\\u005c\xd1\xe7\xbc8$S>6NU\xbd\xc8\x08!\xbfk9\xca\xbe\xeePi\xbe3\x12\x0b?l\x898=\x10\xca\x90?\xce\xcdq\xbes-\xd3>\xfed\xd4\xbe\xa1\xbe\x17>\xc6Rh\xbefv\x01=\x0b\xdf\xea;\x92\x19\\u0000?\x81Q\xf5>r\x9c\x82?\xb4\x16\x8d\xbd\x1dp\x91\xbdw\x88\xb8\xbd\xbc\xf7\xff\xbd?!\x1b\xbf/\xb5\xc0<I\xfd>=\x1f\xf8\x10=qt;\xbd\xb4\x82\xe0>n\x10\xb1\xbe\x81$\x96>_\x10\x88\xbe\xf1\x8a >\xa5\x8c\xb0=1B\xfd\xbcr\xb1\xa0\xbe\x81\x991<1\xdb\xdb=w\xe18?\xcar\x82\xbcm\xaf\xeb<\xbe]\xa9\xbd\xa1\xc1C\xbd\np19\ng7\ntp20\nRp21\ntp22\nbag2\n(g3\n(I0\ntp23\ng9\ntp24\nRp25\n(I1\n(I20\ntp26\ng16\nI00\ng5\n(V\x9f\x18\x0e?\xbc\xd2\x1d\xbf\xeb`\\u0000>FI4>\xf4\xa1\\u001a\xc0`\xe2\x0f\xbf7\xd1\xcf\xbe\xd7\x9c\x88\xbe)x\xeb>I \x8b\xbf\x9e\xc6\x89?\xd3\x9d\x0b\xc0:%\x93\xbf\x0b\xe4\xff\xbdx\\u000a\x0c>3j\xb2>\'\xf7O?w]d\xbf\xe0\x84N\xbe\xc1B\x81?\np27\ng7\ntp28\nRp29\ntp30\nbaa(lp31\ng2\n(g3\n(I0\ntp32\ng9\ntp33\nRp34\n(I1\n(I20\nI6\ntp35\ng16\nI00\ng5\n(V.v\x18>3\xf0\xa0\xbfN\xb2\x08\xbdR\xd7\x96\xbe\xce{\x89\xbf\xf2\xa43>\xb6\xc1\\u000a\xbf\x94\xf9\x9f\xbd8\xfaB?\xb4u\x9c\xbe\xc7\xe6\xbd=\x9f\xbb\x8b>\xbca->Q0\xaa?\xab\xc3C\xbel\xcd\x80\xbe\x14\x89\x81\xbf4\x9dl\xbe\x16\x94Z\xbf\xe9\xd7\x02?\xdd\xdb\x0b\xbe\x83C\x95\xbe\x92\xf6\x18\xbfm\xb5\x88\xbe}+\xa6\xbc\t22>\xe8$\\u000a\xbc\xc1\xb6\xbb\xbf\x99\x98\x05@\xcf\xfc\xc3\xbd=N\xfd\xbdE\xf1\x92\xbe\xa3;\x13\xbf\xab]\xe9\xbd\xe0\xba==\xe3\x8b\xb5\xbc\x06x\x87\xbe[\xad\xe4\xbe\x8ai\xa4<\x0e\xfbJ?\xa7@\xeb?GJ\xb6\xbd\xaf\x11\xf0\xbc(S\x8a\xbf;a\x0e>[\x8b\xd2>\x01\x18\x84\xbf\x9e\xa1\xf2<j(^>\xde\'\x11>\xa8\x03f=N\xe1j?\x9d_\x12?\xa8\xc6\x10?;\x8b\x0c>8d\x8d>Sb\x1c\xbee@\x1e\xc0\x05\xd09?\xebq\xb4\xbd\xd70\xf4\xbc\xaeA\\u001a=\t\x07\x9a\xbe\xd0\\u005c\xe3>\x04\xe5\x05?+g\xe5>(\x0b\xd6<BF\xb7<\x9b\xc1\xe69(\xc2P\xbf\xa5\x1b\\u000a\xc0\xdd\x8dn>\xdf\x9a\xbe\xbf\xdc\xc8\xa5?\x17C\xbb<\xe2P\xd2=bK\x91\xbe\x0b\xa0\x97>\xf5\xa3\xe5=\x1c\xdd\x8c\xbe\xcf4R?\xa8A??\x90a\x17\xbd(k$=\xdb\x82\xb3>\xbb<l?g\x90\x05?GJ\x05\xbf\x0f+x\xbf6m\x15\xbeK\x0b7\xbeq\xdc8=\x8cb0=E\xe1A?\xc7\x8e\xe2?Qe&\xbe\xe8\xd6L@\xe5}\x1b\xc0|\xadI\xbd\x11\x1c\xfe\xbd\xd6r!\xbd\xa8\x97\x0e\xbc\xab%?\xc0\xb4_5\xc0\xf4`<\xbdzNQ>\xf8\xdc\\u000a>\xbf\xbe\xe1\xbd\xdc\xaf\xb4\xbd\xd9C\xaa?M\\u000d\xb2>\xbfl\xfc=U\\u000a\x1f=\xac\xb5\x07?\xbca4?\xb5\x87\xe9?\xc9\xc0\xfd=G\x02\xd7>\xd3\xc1\x98\xbc\xaa$!\xbc\np36\ng7\ntp37\nRp38\ntp39\nbag2\n(g3\n(I0\ntp40\ng9\ntp41\nRp42\n(I1\n(I6\ntp43\ng16\nI00\ng5\n(V+\x96W<\xf0\xc5d>\xe8\x076>(\x0b\x1b>kk\x8a\xbe\xe6\xca\x13=\np44\ng7\ntp45\nRp46\ntp47\nbaa.'

deserialized = pickle.loads(serialized)

print(deserialized)


[[array([[-1.53347790e-01,  1.93725348e-01, -1.34143800e-01,
         6.06588870e-02,  1.15449697e-01, -1.96238130e-01,
         1.21265054e-01,  4.45415467e-01, -1.15695763e+00,
         3.19906652e-01,  3.49047810e-01,  6.81858063e-02,
        -1.69943869e-01, -5.25030233e-02, -1.77881464e-01,
         3.18547815e-01, -7.21922576e-01,  6.91537634e-02,
        -1.72778592e-01,  1.90909028e-01,  6.43650517e-02,
         2.32634753e-01, -1.39359221e-01,  2.49972954e-01,
        -4.62486856e-02,  1.83688641e-01, -1.68406382e-01,
         7.52187446e-02,  2.86563694e-01,  1.14444174e-01],
       [-2.77907789e-01,  9.95855704e-02,  2.47451678e-01,
         2.03887314e-01,  9.94177759e-02, -6.20349824e-01,
         6.22073822e-02, -3.93385977e-01,  1.70611162e-02,
        -1.90580517e-01,  3.65256518e-02, -2.49313787e-01,
         1.74617335e-01,  6.93550259e-02,  1.83161452e-01,
        -6.53279305e-01, -3.38830560e-01,  1.05029859e-01,
         1.46417156e-01,  2.41734698e-01, -2.87395902

In [28]:
debug = [
    0.3108125, 0.3878888888888889, -0.282, 0.308, -0.12777777777777777, 0.2561875, 0.822, 
    0.2586875, 0.32622222222222225, -0.297, 0.242, -0.2833333333333333, 0.2561875, 0.822, 
    0.4165, 0.3972222222222222, -0.119, 0.178, -0.3111111111111111, 0.2561875, 0.822, 
    0.1980625, 0.2788888888888889, -0.344, 0.116, -0.5333333333333333, 0.2561875, 0.822
]


In [204]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

# Get data

w = np.transpose(deserialized[0][0])
b = np.transpose(deserialized[0][1])

sample_id = 10
sample = inputs[sample_id]

print('w is', np.shape(w))
print('sample is', np.shape(sample))

l1 = np.dot(w, sample) + b

printmd ('**manual - layer 1, pre-activation:**')
print(l1)

result = [sigmoid(x) for x in l1]

printmd ('**manual - layer 1, output:**')
print(result)

w = np.transpose(deserialized[1][0])
b = np.transpose(deserialized[1][1])

print('w is', np.shape(w))

l2 = np.dot(w, result) + b

print(l2)

# Tensorflow results

p = model.predict(np.array(inputs)[sample_id:sample_id + 1])
printmd('**tensorflow - output**')
print(p[0])
print(p[0][0] * 1000)
print(p[0][1] * 1000)
print(p[0][2] * 200)
print(p[0][3] * 1000)
print(p[0][4] * 1000)
print(p[0][5] * 200)

#pod1 = Pod(Point(p))

layer_output = model.layers[0].output

intermediate_model = tf.keras.models.Model(inputs = model.input, outputs = layer_output)

intermediate_prediction = intermediate_model.predict(np.array(inputs)[sample_id:sample_id + 1])

printmd('**tensorflow  - layer 1, output**')
print(intermediate_prediction)




w is (20, 28)
sample is (28,)


**manual - layer 1, pre-activation:**

[-0.00334713  0.38992725  0.45333406  0.3322857  -0.15615816  0.51716871
 -0.27932206  0.42670571  0.51259093  0.06551855  1.16183532  0.33996762
 -0.64195453  0.02812562 -0.5407886   0.20596988 -0.59775183  1.71661293
  0.3918293   0.24064428]


**manual - layer 1, output:**

[0.49916321704680106, 0.5962651857441623, 0.6114316434746013, 0.5823154176376595, 0.4610395987984918, 0.6264854770804469, 0.4306199894100857, 0.6050867476507197, 0.6254136522333735, 0.516373781871838, 0.7616660428792555, 0.5841826566382471, 0.3448048486769166, 0.507030942457415, 0.36800415331429287, 0.5513111987586075, 0.3548582076766027, 0.8476920463090961, 0.5967229875584363, 0.5598724169070156]
w is (6, 20)
[ 0.61586041 -0.06720676  0.70557835  0.36765557  0.2188203   0.73103741]


**tensorflow - output**

[ 0.61586034 -0.06720666  0.7055783   0.36765558  0.21882021  0.73103744]
615.8603429794312
-67.20665842294693
141.11566543579102
367.65557527542114
218.8202142715454
146.2074875831604


**tensorflow  - layer 1, output**

[[0.49916324 0.5962652  0.61143166 0.5823154  0.4610396  0.62648547
  0.43061998 0.60508674 0.62541366 0.51637375 0.76166606 0.5841826
  0.34480485 0.5070309  0.3680042  0.5513112  0.35485822 0.847692
  0.59672296 0.5598724 ]]


In [235]:
print(frames[10])
print()
print(inputs[10])


Frame(pods=[Pod(position=Point(x=0.332, y=0.6406666666666667), speed=Point(x=0.6694117647058824, y=0.015294117647058824), angle=0.21666666666666667, waypoint=Point(x=0.593875, y=0.48688888888888887)), Pod(position=Point(x=0.439375, y=0.7045555555555556), speed=Point(x=0.7647058823529411, y=-0.6058823529411764), angle=-0.2111111111111111, waypoint=Point(x=0.593875, y=0.48688888888888887)), Pod(position=Point(x=0.2825, y=0.6996666666666667), speed=Point(x=0.37058823529411766, y=-0.19647058823529412), angle=-0.12222222222222222, waypoint=Point(x=0.593875, y=0.48688888888888887)), Pod(position=Point(x=0.3453125, y=0.7748888888888888), speed=Point(x=0.7447058823529412, y=-0.17294117647058824), angle=-0.07222222222222222, waypoint=Point(x=0.593875, y=0.48688888888888887))], actions=[Action(aim=Point(x=0.9190819888409161, y=0.39406636216280383), power=0.0), Action(aim=Point(x=0.9622746177836934, y=-0.2720800616900231), power=1.0)])

[0.332, 0.6406666666666667, 0.6694117647058824, 0.0152941176